In [1]:
import numpy as np
import pandas as pd
import yaml
import sys
import os
import pickle
from glob import glob
from pathlib import Path
import pickle
import warnings
warnings.filterwarnings('ignore')

sys.path.append('/kaggle/src')
from utils.xgb import fit_xgb
from utils.metric import compute_comptetition_metric
from utils.postprocess import post_process
from utils.set_seed import seed_base
from feature_engineering.stage1 import generate_1st_stage_features

PACKAGE_DIR = Path("/kaggle/src")
CFG = yaml.safe_load(open(PACKAGE_DIR / "config.yaml", "r"))
print(CFG["1st_stage"]["execution"]["exp_id"])

CFG["output_dir"] = f"/kaggle/output/{CFG['1st_stage']['execution']['exp_id']}"
!rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

seed_base(CFG["env"]["seed"])

2023-10-07 12:05:49.291131: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-07 12:05:49.379387: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-07 12:05:49.851510: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/u

debug_002
rm: cannot remove '/kaggle/output/debug_002': No such file or directory


In [2]:
# 特徴生成
files = glob(f"{CFG['dataset']['step_csv_dir']}/*.csv")
train, features = generate_1st_stage_features(files)

# cv splitとマージ
cv_split = pd.read_csv(CFG['dataset']['cv_split_path'])
train["fold"] = train["series_id"].map(cv_split.set_index("series_id")["fold"])
display(train.head(5))
print(train.shape)

generate features: 100%|██████████| 277/277 [01:28<00:00,  3.13it/s]


,anglez_diff_abs,anglez_diff_abs_rolling_max_10,anglez_diff_abs_rolling_max_100,anglez_diff_abs_rolling_max_1000,anglez_diff_abs_rolling_max_50,anglez_diff_abs_rolling_mean_10,anglez_diff_abs_rolling_mean_100,anglez_diff_abs_rolling_mean_1000,anglez_diff_abs_rolling_mean_50,anglez_diff_abs_rolling_std_10,...,enmo_rolling_mean_1000,enmo_rolling_mean_50,enmo_rolling_std_10,enmo_rolling_std_100,enmo_rolling_std_1000,enmo_rolling_std_50,target,step,series_id,fold
0,0.006461,0.043200,NaN,NaN,NaN,0.007771,NaN,NaN,NaN,0.015468,...,NaN,NaN,0.000000,NaN,NaN,NaN,1,11.5,af91d9a50547,3
1,0.000117,0.000833,NaN,NaN,0.047400,0.000192,NaN,NaN,0.002144,0.000324,...,NaN,0.000000,0.000000,NaN,NaN,0.000000,1,35.5,af91d9a50547,3
2,0.000767,0.005475,0.5969,NaN,0.457854,0.001198,0.022684,NaN,0.024919,0.001924,...,NaN,0.000237,0.000000,0.00146,NaN,0.001634,1,59.5,af91d9a50547,3
3,0.083833,0.347392,0.5969,NaN,0.596900,0.083032,0.025412,NaN,0.042692,0.115663,...,NaN,0.000300,0.001924,0.00146,NaN,0.002064,1,83.5,af91d9a50547,3
4,0.014787,0.048846,0.5969,NaN,0.309038,0.012615,0.028460,NaN,0.028898,0.018788,...,NaN,0.000088,0.000000,0.00146,NaN,0.000604,1,107.5,af91d9a50547,3


(5331163, 54)


In [3]:
# 学習
trn_oof, models = fit_xgb(
    X=train, 
    y=train["target"], 
    folds=train["fold"].astype(int), 
    features=features.all_features(),
    params=CFG["1st_stage"]["xgboost"], 
    es_rounds=100,
)
train["oof"] = trn_oof
train = train.drop(columns=features.all_features())
train = train.sort_values(["series_id", "step"]).reset_index(drop=True)

# 保存
for i, model in enumerate(models):
    model.save_model(os.path.join(CFG["output_dir"], f'xgb_fold{i}.model'))
with open(os.path.join(CFG["output_dir"], "features.pkl"), "wb") as f:
    pickle.dump(features, f)
train.to_csv(os.path.join(CFG["output_dir"], "oof.csv"), index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

== fold 0 ==
[0]	eval-logloss:0.61808
[100]	eval-logloss:0.13800
[200]	eval-logloss:0.13598
[300]	eval-logloss:0.13587
[330]	eval-logloss:0.13616


 20%|██        | 1/5 [05:03<20:14, 303.55s/it]

== fold 1 ==
[0]	eval-logloss:0.61756
[100]	eval-logloss:0.11212
[200]	eval-logloss:0.10559
[300]	eval-logloss:0.10371
[400]	eval-logloss:0.10332
[500]	eval-logloss:0.10346
[517]	eval-logloss:0.10367


 40%|████      | 2/5 [12:46<19:52, 397.45s/it]

...

 60%|██████    | 3/5 [21:13<14:54, 447.38s/it]

== fold 3 ==
[0]	eval-logloss:0.61825
[100]	eval-logloss:0.13984
[200]	eval-logloss:0.13835
[299]	eval-logloss:0.13859


 80%|████████  | 4/5 [25:54<06:21, 381.68s/it]

== fold 4 ==
[0]	eval-logloss:0.61773
[100]	eval-logloss:0.13438
[200]	eval-logloss:0.13065
[300]	eval-logloss:0.12994
[400]	eval-logloss:0.12946
[500]	eval-logloss:0.12933
[600]	eval-logloss:0.12943
[669]	eval-logloss:0.12963


100%|██████████| 5/5 [36:06<00:00, 433.25s/it]


In [4]:
# 後処理
sub = post_process(train)
sub.to_csv(os.path.join(CFG["output_dir"], "submission.csv"), index=False)

# スコア計算
labels = pd.read_csv(f"{CFG['dataset']['competition_dir']}/train_events.csv").dropna()
score, ap_table = compute_comptetition_metric(labels, sub)
print(f"score: {score:.4f}")
display(ap_table)

post process: 100%|██████████| 277/277 [00:46<00:00,  6.01it/s]


score: 0.3956


event   tolerance
onset   12           0.003089
        36           0.027864
        60           0.089549
        90           0.183422
        120          0.354996
        150          0.459382
        180          0.564502
        240          0.673075
        300          0.720683
        360          0.743531
wakeup  12           0.004022
        36           0.041564
        60           0.120443
        90           0.266414
        120          0.369687
        150          0.505100
        180          0.578235
        240          0.692695
        300          0.745094
        360          0.768475
dtype: float64